In [1]:
from transformers import AutoProcessor, LlavaForConditionalGeneration
import numpy as np
from utils import *
import glob
import json
import datasets as ds
import os

image_paths = glob.glob('/data/maryam.sana/Uni-ControlNet/data/images/*.png')
model = LlavaForConditionalGeneration.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

/data/maryam.sana/anaconda3/envs/compressai/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-06-13 21:08:07.805886: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-13 21:08:07.851309: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 21:08:10.926042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warni

In [2]:
def get_caption(frame):
    prompt = "<image>\nUSER:Give a detailed visual description of this image ?\nASSISTANT:"
    inputs = processor(text=prompt, images=frame, return_tensors="pt")
    generate_ids = model.generate(**inputs, max_length=200)
    caption_llava = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    return caption_llava.split('ASSISTANT:')[1]  

In [22]:
import cv2
import os

def check_images(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                try:
                    img = cv2.imread(os.path.join(root, file))
                    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    # print(image.shape, os.path.join(root, file))
                    if img is None:
                        print(f"Failed to read image: {file}")
                except Exception as e:
                    print(f"Error reading image {file}: {e}")

check_images('data/images')

In [18]:
# from PIL import Image

# anno_file = '/data/maryam.sana/Uni-ControlNet/data/data.txt'

# with open(anno_file, 'w') as anno:
#     for i, img_path in enumerate(image_paths[0:5]):
#         # Load image
#         frame = Image.open(img_path)
        
#         # Generate caption for the image
#         caption = get_caption(frame)
#         caption = " ".join(caption.split())
#         # print(caption)
        
#         # Get the file ID (assuming it's the filename without the extension)
#         file_id = os.path.splitext(os.path.basename(img_path))[0]
#         print(f"{file_id} {caption}\n")
        
#         anno.write(f"{file_id}\t{caption}\n")

# print(f"Annotation file {anno_file} has been created.")
  

In [24]:
import os

def read_processed_files(data_file):
    processed_files = set()
    with open(data_file, 'r') as file:
        for line in file:
            file_id, caption = line.strip().split('\t')
            processed_files.add(file_id)
    return processed_files

def get_all_png_files(directory):
    png_files = set()
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.png'):
                file_id = os.path.splitext(file)[0]  # Remove the file extension
                png_files.add(file_id)
    return png_files

def find_unprocessed_files(png_directory, data_file):
    processed_files = read_processed_files(data_file)
    all_png_files = get_all_png_files(png_directory)
    unprocessed_files = all_png_files - processed_files
    return unprocessed_files

if __name__ == "__main__":
    png_directory = 'data/images/'
    data_file = './data/data.txt'
    file_list = []
    
    unprocessed_files = find_unprocessed_files(png_directory, data_file)
    if unprocessed_files:
        print("Unprocessed files:")
        for file in unprocessed_files:
            file_list.append(file)
            
            # print(f"{file}.png")
    else:
        print("All files are processed.")


Unprocessed files:


In [25]:
import glob

len(glob.glob('data/images/*.png'))

147

In [26]:
len(file_list)

73

In [27]:
file_list

['j_032',
 'r_036',
 'y_038',
 'y_044',
 'j_002',
 'r_039',
 'j_060',
 'r_019',
 'r_048',
 'r_017',
 'y_020',
 'y_053',
 'j_001-checkpoint',
 'j_005',
 'r_041',
 's_022',
 'r_059',
 'j_011',
 'r_007',
 'r_045',
 'r_008',
 'y_058',
 'r_054',
 'j_050',
 'j_026',
 'r_029',
 'j_020',
 'j_003',
 'r_046',
 'j_055',
 'j_021',
 's_029',
 'j_034',
 'j_025',
 'j_001',
 'y_039',
 'r_050',
 'y_054',
 'j_044',
 'r_005',
 'y_023',
 'y_048',
 's_001',
 'y_049',
 'r_052',
 'y_015',
 'j_049',
 'y_007',
 'y_009',
 'j_059',
 's_026',
 'j_043',
 'r_060',
 'y_047',
 's_014',
 's_020',
 'j_036',
 'j_045',
 'r_030',
 'j_013',
 'y_011',
 'j_023',
 'r_006',
 'j_046',
 'r_049',
 'y_059',
 'r_021',
 'j_002-checkpoint',
 'y_042',
 'y_040',
 'y_046',
 'r_058',
 'y_043']